In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

In [3]:
from numpy.core.fromnumeric import shape
def load_data(path,batch_size,input_size):
    
    normalize = transforms.Compose([
        transforms.Resize((227,227)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ]) 
    transform_dict = {"src":  normalize}  
    # train_path=path+"/train"
    # test_path=path+"/test"
    data = datasets.ImageFolder(root=path,transform=transform_dict["src"])
    # transform_dict1 = {"test":  normalize} 
    # test1 = datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # transform_dict = {"test":  normalize}
    # test= datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # train_size = int((1- (test_split + val_split)) * len(data))
    # test_size = int((1 - (val_split)) * len(data)) - train_size
    # val_size = len(data) - train_size - test_size
    train_size=int(0.75*len(data))
    test_size=int(len(data)-train_size)
    train, test = td.random_split(data,[train_size,test_size])

    data_loader_train = td.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    data_loader_test = td.DataLoader(test,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    # data_loader_val = td.DataLoader(val,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    return data_loader_train, data_loader_test

In [4]:
data_loader_train,data_loader_test=load_data(r"C:\Users\Admin\Desktop\COMP 6721\Project\datasets\Dataset1",32,64)

In [5]:
data_loader_test.batch_size

32

In [13]:
# img2=image.imread(r"C:\Users\Admin\Desktop\COMP 6721\Project\datasets\Dataset3\train\1.2.826.0.1.3680043.8.498.10144276773552601236124857108161469299-c.png")
# img2.shape
# #                   1.2.826.0.1.3680043.8.498.10299385524344582994601970314833752000-c.png")
# # plt.imshow(img2)

In [14]:


class AlexNet(torch.nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        return logits

In [15]:
import torch.optim as optim
net = AlexNet(4).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [16]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
net.to(device)
acc_array = []
num_epochs = 30
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
        # print("kjnfjnrnkrn",i)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            accuracy = (correct / total) * 100
            acc_array.append(float(accuracy))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/30], Step [10/497], Loss: 1.1097, Accuracy: 53.12%
Epoch [1/30], Step [20/497], Loss: 1.1852, Accuracy: 56.25%
Epoch [1/30], Step [30/497], Loss: 0.9803, Accuracy: 65.62%
Epoch [1/30], Step [40/497], Loss: 1.2921, Accuracy: 43.75%
Epoch [1/30], Step [50/497], Loss: 1.2340, Accuracy: 53.12%
Epoch [1/30], Step [60/497], Loss: 1.0896, Accuracy: 56.25%
Epoch [1/30], Step [70/497], Loss: 1.2237, Accuracy: 46.88%
Epoch [1/30], Step [80/497], Loss: 1.2838, Accuracy: 37.50%
Epoch [1/30], Step [90/497], Loss: 1.1380, Accuracy: 59.38%
Epoch [1/30], Step [100/497], Loss: 1.1916, Accuracy: 53.12%
Epoch [1/30], Step [110/497], Loss: 1.2961, Accuracy: 37.50%
Epoch [1/30], Step [120/497], Loss: 1.1366, Accuracy: 53.12%
Epoch [1/30], Step [130/497], Loss: 1.0741, Accuracy: 50.00%
Epoch [1/30], Step [140/497], Loss: 1.0803, Accuracy: 59.38%
Epoch [1/30], Step [150/497], Loss: 1.2920, Accuracy: 40.62%
Epoch [1/30], Step [160/497], Loss: 1.1429, Accuracy: 53.12%
Epoch [1/30], Step

Epoch [3/30], Step [380/497], Loss: 1.1856, Accuracy: 46.88%
Epoch [3/30], Step [390/497], Loss: 1.2071, Accuracy: 59.38%
Epoch [3/30], Step [400/497], Loss: 1.1819, Accuracy: 43.75%
Epoch [3/30], Step [410/497], Loss: 1.3335, Accuracy: 40.62%
Epoch [3/30], Step [420/497], Loss: 1.3944, Accuracy: 34.38%
Epoch [3/30], Step [430/497], Loss: 1.1880, Accuracy: 50.00%
Epoch [3/30], Step [440/497], Loss: 1.2253, Accuracy: 43.75%
Epoch [3/30], Step [450/497], Loss: 1.2136, Accuracy: 34.38%
Epoch [3/30], Step [460/497], Loss: 1.0752, Accuracy: 56.25%
Epoch [3/30], Step [470/497], Loss: 1.1715, Accuracy: 53.12%
Epoch [3/30], Step [480/497], Loss: 1.2298, Accuracy: 50.00%
Epoch [3/30], Step [490/497], Loss: 1.1889, Accuracy: 37.50%
Epoch [4/30], Step [10/497], Loss: 1.1595, Accuracy: 59.38%
Epoch [4/30], Step [20/497], Loss: 1.1248, Accuracy: 50.00%
Epoch [4/30], Step [30/497], Loss: 1.2143, Accuracy: 46.88%
Epoch [4/30], Step [40/497], Loss: 1.1126, Accuracy: 59.38%
Epoch [4/30], Step [50/497],

Epoch [6/30], Step [260/497], Loss: 1.0858, Accuracy: 50.00%
Epoch [6/30], Step [270/497], Loss: 0.9967, Accuracy: 68.75%
Epoch [6/30], Step [280/497], Loss: 1.2384, Accuracy: 40.62%
Epoch [6/30], Step [290/497], Loss: 1.3068, Accuracy: 40.62%
Epoch [6/30], Step [300/497], Loss: 1.1690, Accuracy: 50.00%
Epoch [6/30], Step [310/497], Loss: 1.1110, Accuracy: 59.38%
Epoch [6/30], Step [320/497], Loss: 1.2480, Accuracy: 40.62%
Epoch [6/30], Step [330/497], Loss: 1.3127, Accuracy: 31.25%
Epoch [6/30], Step [340/497], Loss: 1.2933, Accuracy: 43.75%
Epoch [6/30], Step [350/497], Loss: 1.2542, Accuracy: 50.00%
Epoch [6/30], Step [360/497], Loss: 1.2313, Accuracy: 37.50%
Epoch [6/30], Step [370/497], Loss: 1.3429, Accuracy: 43.75%
Epoch [6/30], Step [380/497], Loss: 1.2756, Accuracy: 40.62%
Epoch [6/30], Step [390/497], Loss: 1.2020, Accuracy: 46.88%
Epoch [6/30], Step [400/497], Loss: 1.1309, Accuracy: 56.25%
Epoch [6/30], Step [410/497], Loss: 1.2383, Accuracy: 43.75%
Epoch [6/30], Step [420/

Epoch [9/30], Step [140/497], Loss: 1.0318, Accuracy: 68.75%
Epoch [9/30], Step [150/497], Loss: 1.3773, Accuracy: 46.88%
Epoch [9/30], Step [160/497], Loss: 1.1683, Accuracy: 50.00%
Epoch [9/30], Step [170/497], Loss: 1.1545, Accuracy: 56.25%
Epoch [9/30], Step [180/497], Loss: 1.1182, Accuracy: 50.00%
Epoch [9/30], Step [190/497], Loss: 1.2163, Accuracy: 37.50%
Epoch [9/30], Step [200/497], Loss: 1.1980, Accuracy: 25.00%
Epoch [9/30], Step [210/497], Loss: 1.2280, Accuracy: 43.75%
Epoch [9/30], Step [220/497], Loss: 1.2022, Accuracy: 46.88%
Epoch [9/30], Step [230/497], Loss: 1.1832, Accuracy: 46.88%
Epoch [9/30], Step [240/497], Loss: 1.2612, Accuracy: 40.62%
Epoch [9/30], Step [250/497], Loss: 1.1892, Accuracy: 53.12%
Epoch [9/30], Step [260/497], Loss: 1.2202, Accuracy: 53.12%
Epoch [9/30], Step [270/497], Loss: 1.0702, Accuracy: 62.50%
Epoch [9/30], Step [280/497], Loss: 1.2058, Accuracy: 50.00%
Epoch [9/30], Step [290/497], Loss: 1.2658, Accuracy: 53.12%
Epoch [9/30], Step [300/

Epoch [12/30], Step [10/497], Loss: 1.1733, Accuracy: 37.50%
Epoch [12/30], Step [20/497], Loss: 1.1342, Accuracy: 53.12%
Epoch [12/30], Step [30/497], Loss: 1.2207, Accuracy: 46.88%
Epoch [12/30], Step [40/497], Loss: 1.1291, Accuracy: 53.12%
Epoch [12/30], Step [50/497], Loss: 1.0702, Accuracy: 56.25%
Epoch [12/30], Step [60/497], Loss: 1.1431, Accuracy: 56.25%
Epoch [12/30], Step [70/497], Loss: 1.2174, Accuracy: 50.00%
Epoch [12/30], Step [80/497], Loss: 1.2040, Accuracy: 37.50%
Epoch [12/30], Step [90/497], Loss: 1.2224, Accuracy: 43.75%
Epoch [12/30], Step [100/497], Loss: 1.2216, Accuracy: 37.50%
Epoch [12/30], Step [110/497], Loss: 1.2118, Accuracy: 37.50%
Epoch [12/30], Step [120/497], Loss: 1.1194, Accuracy: 53.12%
Epoch [12/30], Step [130/497], Loss: 1.0470, Accuracy: 62.50%
Epoch [12/30], Step [140/497], Loss: 1.1169, Accuracy: 59.38%
Epoch [12/30], Step [150/497], Loss: 1.2253, Accuracy: 34.38%
Epoch [12/30], Step [160/497], Loss: 1.1819, Accuracy: 40.62%
Epoch [12/30], St

...

Epoch [20/30], Step [80/497], Loss: 1.2020, Accuracy: 50.00%
Epoch [20/30], Step [90/497], Loss: 1.1900, Accuracy: 46.88%
Epoch [20/30], Step [100/497], Loss: 1.0262, Accuracy: 53.12%
Epoch [20/30], Step [110/497], Loss: 1.2393, Accuracy: 56.25%
Epoch [20/30], Step [120/497], Loss: 1.2184, Accuracy: 53.12%
Epoch [20/30], Step [130/497], Loss: 1.3101, Accuracy: 40.62%
Epoch [20/30], Step [140/497], Loss: 1.3433, Accuracy: 37.50%
Epoch [20/30], Step [150/497], Loss: 1.2101, Accuracy: 40.62%
Epoch [20/30], Step [160/497], Loss: 1.3293, Accuracy: 31.25%
Epoch [20/30], Step [170/497], Loss: 1.0855, Accuracy: 43.75%
Epoch [20/30], Step [180/497], Loss: 1.3561, Accuracy: 40.62%
Epoch [20/30], Step [190/497], Loss: 1.0873, Accuracy: 46.88%
Epoch [20/30], Step [200/497], Loss: 1.2269, Accuracy: 43.75%
Epoch [20/30], Step [210/497], Loss: 1.0887, Accuracy: 50.00%
Epoch [20/30], Step [220/497], Loss: 1.2534, Accuracy: 43.75%
Epoch [20/30], Step [230/497], Loss: 1.0451, Accuracy: 59.38%
Epoch [20/

Epoch [22/30], Step [430/497], Loss: 1.3535, Accuracy: 37.50%
Epoch [22/30], Step [440/497], Loss: 1.2711, Accuracy: 31.25%
Epoch [22/30], Step [450/497], Loss: 1.0490, Accuracy: 46.88%
Epoch [22/30], Step [460/497], Loss: 1.1241, Accuracy: 53.12%
Epoch [22/30], Step [470/497], Loss: 1.0585, Accuracy: 46.88%
Epoch [22/30], Step [480/497], Loss: 1.1754, Accuracy: 56.25%
Epoch [22/30], Step [490/497], Loss: 1.1674, Accuracy: 53.12%
Epoch [23/30], Step [10/497], Loss: 1.1878, Accuracy: 43.75%
Epoch [23/30], Step [20/497], Loss: 1.2782, Accuracy: 37.50%
Epoch [23/30], Step [30/497], Loss: 1.2288, Accuracy: 50.00%
Epoch [23/30], Step [40/497], Loss: 1.1072, Accuracy: 50.00%
Epoch [23/30], Step [50/497], Loss: 1.1483, Accuracy: 50.00%
Epoch [23/30], Step [60/497], Loss: 1.1193, Accuracy: 53.12%
Epoch [23/30], Step [70/497], Loss: 1.0423, Accuracy: 59.38%
Epoch [23/30], Step [80/497], Loss: 1.3847, Accuracy: 31.25%
Epoch [23/30], Step [90/497], Loss: 1.1301, Accuracy: 53.12%
Epoch [23/30], St

Epoch [25/30], Step [290/497], Loss: 1.2235, Accuracy: 50.00%
Epoch [25/30], Step [300/497], Loss: 1.2294, Accuracy: 37.50%
Epoch [25/30], Step [310/497], Loss: 1.2953, Accuracy: 34.38%
Epoch [25/30], Step [320/497], Loss: 1.2434, Accuracy: 37.50%
Epoch [25/30], Step [330/497], Loss: 1.3171, Accuracy: 37.50%
Epoch [25/30], Step [340/497], Loss: 1.1888, Accuracy: 50.00%
Epoch [25/30], Step [350/497], Loss: 1.1842, Accuracy: 46.88%
Epoch [25/30], Step [360/497], Loss: 1.2999, Accuracy: 40.62%
Epoch [25/30], Step [370/497], Loss: 1.1449, Accuracy: 56.25%
Epoch [25/30], Step [380/497], Loss: 1.1628, Accuracy: 37.50%
Epoch [25/30], Step [390/497], Loss: 1.0481, Accuracy: 53.12%
Epoch [25/30], Step [400/497], Loss: 1.1795, Accuracy: 46.88%
Epoch [25/30], Step [410/497], Loss: 1.2343, Accuracy: 43.75%
Epoch [25/30], Step [420/497], Loss: 1.1345, Accuracy: 53.12%
Epoch [25/30], Step [430/497], Loss: 1.0454, Accuracy: 56.25%
Epoch [25/30], Step [440/497], Loss: 1.1868, Accuracy: 59.38%
Epoch [2

Epoch [28/30], Step [150/497], Loss: 1.3415, Accuracy: 46.88%
Epoch [28/30], Step [160/497], Loss: 1.2024, Accuracy: 46.88%
Epoch [28/30], Step [170/497], Loss: 1.2109, Accuracy: 53.12%
Epoch [28/30], Step [180/497], Loss: 0.9683, Accuracy: 65.62%
Epoch [28/30], Step [190/497], Loss: 1.2640, Accuracy: 46.88%
Epoch [28/30], Step [200/497], Loss: 1.1446, Accuracy: 43.75%
Epoch [28/30], Step [210/497], Loss: 1.1864, Accuracy: 40.62%
Epoch [28/30], Step [220/497], Loss: 1.2638, Accuracy: 37.50%
Epoch [28/30], Step [230/497], Loss: 1.0246, Accuracy: 59.38%
Epoch [28/30], Step [240/497], Loss: 1.2790, Accuracy: 50.00%
Epoch [28/30], Step [250/497], Loss: 0.9672, Accuracy: 65.62%
Epoch [28/30], Step [260/497], Loss: 1.2879, Accuracy: 43.75%
Epoch [28/30], Step [270/497], Loss: 1.2754, Accuracy: 40.62%
Epoch [28/30], Step [280/497], Loss: 1.0688, Accuracy: 53.12%
Epoch [28/30], Step [290/497], Loss: 1.2227, Accuracy: 43.75%
Epoch [28/30], Step [300/497], Loss: 1.2494, Accuracy: 43.75%
Epoch [2

######## Training Finished in 2404.0386641025543 seconds ###########


In [17]:
######## Write your code here #############
net.eval() 
count=0
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        # count+=1
        # print(count)
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 5292 test images: 48.41269841269841 %


## Training with Model directly from Pytorch 

In [7]:
import torch.optim as optim

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

Using cache found in C:\Users\Admin/.cache\torch\hub\pytorch_vision_v0.10.0


In [9]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 5
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
#         print("kjnfjnrnkrn",i)\\
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/5], Step [10/497], Loss: 6.7191, Accuracy: 50.00%
Epoch [1/5], Step [20/497], Loss: 5.9979, Accuracy: 46.88%
Epoch [1/5], Step [30/497], Loss: 1.5149, Accuracy: 43.75%
Epoch [1/5], Step [40/497], Loss: 1.5791, Accuracy: 34.38%
Epoch [1/5], Step [50/497], Loss: 1.3338, Accuracy: 40.62%
Epoch [1/5], Step [60/497], Loss: 1.2137, Accuracy: 50.00%
Epoch [1/5], Step [70/497], Loss: 1.2648, Accuracy: 43.75%
Epoch [1/5], Step [80/497], Loss: 1.1942, Accuracy: 46.88%
Epoch [1/5], Step [90/497], Loss: 1.3345, Accuracy: 43.75%
Epoch [1/5], Step [100/497], Loss: 1.3920, Accuracy: 40.62%
Epoch [1/5], Step [110/497], Loss: 1.1425, Accuracy: 53.12%
Epoch [1/5], Step [120/497], Loss: 1.2261, Accuracy: 46.88%
Epoch [1/5], Step [130/497], Loss: 1.2162, Accuracy: 37.50%
Epoch [1/5], Step [140/497], Loss: 1.0173, Accuracy: 68.75%
Epoch [1/5], Step [150/497], Loss: 0.9700, Accuracy: 62.50%
Epoch [1/5], Step [160/497], Loss: 1.0369, Accuracy: 65.62%
Epoch [1/5], Step [170/497], Loss:

Epoch [3/5], Step [400/497], Loss: 0.2448, Accuracy: 90.62%
Epoch [3/5], Step [410/497], Loss: 0.3384, Accuracy: 90.62%
Epoch [3/5], Step [420/497], Loss: 0.7493, Accuracy: 68.75%
Epoch [3/5], Step [430/497], Loss: 0.4505, Accuracy: 81.25%
Epoch [3/5], Step [440/497], Loss: 0.5082, Accuracy: 84.38%
Epoch [3/5], Step [450/497], Loss: 0.8902, Accuracy: 59.38%
Epoch [3/5], Step [460/497], Loss: 0.8193, Accuracy: 65.62%
Epoch [3/5], Step [470/497], Loss: 0.3337, Accuracy: 90.62%
Epoch [3/5], Step [480/497], Loss: 0.4018, Accuracy: 78.12%
Epoch [3/5], Step [490/497], Loss: 0.4334, Accuracy: 81.25%
Epoch [4/5], Step [10/497], Loss: 0.6456, Accuracy: 71.88%
Epoch [4/5], Step [20/497], Loss: 0.4619, Accuracy: 81.25%
Epoch [4/5], Step [30/497], Loss: 0.4107, Accuracy: 84.38%
Epoch [4/5], Step [40/497], Loss: 0.6175, Accuracy: 81.25%
Epoch [4/5], Step [50/497], Loss: 0.8013, Accuracy: 65.62%
Epoch [4/5], Step [60/497], Loss: 0.4664, Accuracy: 81.25%
Epoch [4/5], Step [70/497], Loss: 0.5154, Accu

In [12]:
model.eval() 
perds = []
target = []
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        perds.extend(predicted)
        target.extend(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))
    
# plt.plot(Accuracies)

Test Accuracy of the model on the 5292 test images: 77.5132275132275 %


In [13]:
!pip install torchmetrics

     -------------------------------------- 529.7/529.7 kB 6.6 MB/s eta 0:00:00


In [14]:
from torchmetrics.functional import precision_recall
from torchmetrics import F1Score
from torchmetrics import ConfusionMatrix
perds1 = torch.stack(perds)
target1 = torch.stack(target)
f1 = F1Score(num_classes=4).to(device)
print(f1(perds1, target1))
print(precision_recall(perds1, target1, average='macro', num_classes=4))
confmat = ConfusionMatrix(num_classes=4).to(device)
confmat(perds1, target1)

tensor(0.7751, device='cuda:0')
(tensor(0.8440, device='cuda:0'), tensor(0.7099, device='cuda:0'))


tensor([[ 272,  227,  397,    3],
        [   6, 1179,  374,    0],
        [  10,  101, 2392,   20],
        [   1,   38,   13,  259]], device='cuda:0')